In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
import datetime as dt
import re

In [2]:
# Dealers description
dealer_dct = {
    'id': 'simpleautos',
    'name': 'Khushi Auto',
    'street': '2829 Derry Rd E.',
    'city': 'Mississauga',
    'zip': 'L4T1A5',
    'province': 'ON',
    'phone': '905-965-7800',
    'url': 'https://www.simpleautos.com',
    'latitude': '43.701760',
    'longitude': '-79.645490'
}

print(json.dumps(dealer_dct, indent=4))

with open('data2/simpleautos.json', 'w', encoding='utf-8') as fp:
    json.dump(dealer_dct, fp, indent=4)

{
    "id": "simpleautos",
    "name": "Khushi Auto",
    "street": "2829 Derry Rd E.",
    "city": "Mississauga",
    "zip": "L4T1A5",
    "province": "ON",
    "phone": "905-965-7800",
    "url": "https://www.simpleautos.com",
    "latitude": "43.701760",
    "longitude": "-79.645490"
}


In [3]:
base_url ='https://www.simpleautos.ca/'
inventory_url = 'https://www.simpleautos.ca/used-cars'

In [4]:
def parse_inventory(query_args):
    global base_url, inventory_url
    res = []

    time.sleep(1.1)
    response = requests.get(inventory_url, query_args)
    
    soup = BeautifulSoup(response.text, 'lxml')
    vehicles = soup.find_all('div', class_='vehicle search-result-item vehicleList')
    if vehicles is None:
        return res, False
    
    for v in vehicles:
        # check if it isn't sold
        sold = v.find('div', class_='SoldVehicle')
        if sold is not None:
            return res, False # all the rest of the inventory is a list of sold vehicles, no need to go further

        # get vehicle description page url
        url = base_url + v.div.contents[1].div.div.a['href']
        
        # Printing URL (for debugginh purposes)
        print(url)
        res.append(url)
    
    return res, True

In [5]:
# Making a list of inventory URLs
vehicle_urls = []
cpage = 1

urls, has_more = parse_inventory({'ppage': 100, 'cpage': cpage})

while has_more:
    vehicle_urls.extend(urls)
    cpage += 1
    urls, has_more = parse_inventory({'cpage': cpage})
    
vehicle_urls.extend(urls)
print(len(vehicle_urls))

https://www.simpleautos.ca//cars/used/2013-Nissan-Altima-Sedan-114169
https://www.simpleautos.ca//cars/used/2019-Hyundai-Elantra-Sedan-114162
https://www.simpleautos.ca//cars/used/2018-Honda-Civic-Sedan-Sedan-114159
https://www.simpleautos.ca//cars/used/2016-Volkswagen-Passat-Sedan-114156
https://www.simpleautos.ca//cars/used/2018-Toyota-Corolla-Sedan-114153
https://www.simpleautos.ca//cars/used/2017-Honda-Accord-Sedan-Sedan-111421
https://www.simpleautos.ca//cars/used/2017-Honda-Accord-Sedan-Sedan-110440
https://www.simpleautos.ca//cars/used/2010-Ford-F-150-Truck-Extended-Cab-110187
https://www.simpleautos.ca//cars/used/2016-Volkswagen-Jetta-Sedan-Sedan-106068
https://www.simpleautos.ca//cars/used/2017-Hyundai-Sonata-Sedan-106070
https://www.simpleautos.ca//cars/used/2017-Toyota-Camry-Sedan-106069
https://www.simpleautos.ca//cars/used/2017-Honda-Civic-Sedan-Sedan-106118
https://www.simpleautos.ca//cars/used/2008-Ford-Super-Duty-F-250-Truck-Extended-Cab-106056
https://www.simpleautos.c

KeyboardInterrupt: 

In [6]:
# Function parses vehicle description page and returns a dictionary with its parameters
def parse_car_infopage(url):
    # Translation table from their names to our key names
    translation_table = {
        'Make:': 'make',
        'Model:': 'model',
        'Year:': 'year',
        'Body Style:': 'body_style',
        'Odometer:': 'mileage', # Note: special treatment
        'Transmission:': 'transmission',
        'Engine:': 'cylinders', # Note: special treatment
        'Engine Size:': 'displacement',
        'Driveline:': 'drivetrain',
        'Exterior Color:': 'ext_color', 
        'Interior Color:': 'int_color',
        'Doors:': 'doors', # Note: special treatment
        'Passengers:': 'passengers',
        'Stock Number:': 'stock_id',
        'VIN:': 'vin'
    }
    
    # Fetching the page
    time.sleep(1.1)
    response = requests.get(url)

    # Parsing
    soup = BeautifulSoup(response.text, 'lxml')
    price_node = soup.find('span', class_='PriceValue')
    details_node = soup.find('div', class_='VehicleInfoDetails')
    descr_node = soup.find('div', class_='seller_comments')

    # Basic info
    res = {
        'timestamp': dt.datetime.now().isoformat(),
        'dealer': 'simpleautos',
        'url': url
    }

    # Price
    price = re.sub('[^0-9]', '', price_node.contents[0]) # Cleaning junk, leaving only numbers
    res['price'] = price

    # Reading core parameters
    spans = details_node.find_all('span')

    for s in spans:
        if s.text in translation_table:
            key = translation_table[s.text]
            value = s.next_sibling.text

            # Ad Hoc treatment
            if key in ['cylinders', 'doors']:
                value = value.split()[0]
            elif key == 'mileage':
                value = re.sub('[^0-9]', '', value) # Remove all non numbers using regexp substitution

            res[key] = value
    
    # Free text description        
    #res['description'] = descr.find('p').text

    # Printing result (for debugging purposes)
    print(json.dumps(res, indent=4))
    
    # Done
    return res

In [7]:
# Now scraping vehicle data, one by one
vehicle_data = []

for url in vehicle_urls:
    vehicle_data.append(parse_car_infopage(url))
    
print(len(vehicle_data))

with open('data2/simpleautos_cars.json', 'w', encoding='utf-8') as fp:
    json.dump(vehicle_data, fp, indent=4)

{
    "timestamp": "2020-04-25T09:49:20.462933",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2013-Nissan-Altima-Sedan-114169",
    "price": "8499",
    "make": "Nissan",
    "model": "Altima",
    "year": "2013",
    "body_style": "Sedan",
    "mileage": "159208",
    "transmission": "CVT",
    "cylinders": "4",
    "displacement": "2.5",
    "drivetrain": "FWD",
    "ext_color": "White",
    "int_color": "Beige",
    "doors": "4",
    "passengers": "5",
    "stock_id": "291",
    "vin": "1N4AL3AP0DN418291"
}
{
    "timestamp": "2020-04-25T09:49:22.505271",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2019-Hyundai-Elantra-Sedan-114162",
    "price": "17499",
    "make": "Hyundai",
    "model": "Elantra",
    "year": "2019",
    "body_style": "Sedan",
    "mileage": "45106",
    "transmission": "Automatic",
    "cylinders": "4",
    "displacement": "2.0",
    "drivetrain": "FWD",
    "ext_color": "Gray",
    "int_color

{
    "timestamp": "2020-04-25T09:49:59.902749",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2017-Chevrolet-Cruze-Sedan-106054",
    "price": "15999",
    "make": "Chevrolet",
    "model": "Cruze",
    "year": "2017",
    "body_style": "Sedan",
    "mileage": "48837",
    "transmission": "6 Speed Automatic",
    "cylinders": "4",
    "displacement": "1.4",
    "drivetrain": "FWD",
    "ext_color": "White",
    "int_color": "Black",
    "doors": "4",
    "passengers": "5",
    "stock_id": "948",
    "vin": "3G1BE5SM2HS535948"
}
{
    "timestamp": "2020-04-25T09:50:03.666393",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2018-Hyundai-Elantra-Sedan-106095",
    "price": "13499",
    "make": "Hyundai",
    "model": "Elantra",
    "year": "2018",
    "body_style": "Sedan",
    "mileage": "16769",
    "transmission": "Manual",
    "cylinders": "4",
    "displacement": "2.0",
    "drivetrain": "FWD",
    "ext_color": "Blue",

{
    "timestamp": "2020-04-25T09:50:38.434519",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2016-Chevrolet-Cruze-Limited-Sedan-106114",
    "price": "13499",
    "make": "Chevrolet",
    "model": "Cruze Limited",
    "year": "2016",
    "body_style": "Sedan",
    "mileage": "70773",
    "transmission": "Automatic",
    "cylinders": "4",
    "displacement": "1.4",
    "drivetrain": "FWD",
    "ext_color": "Silver",
    "int_color": "-",
    "doors": "4",
    "passengers": "-",
    "stock_id": "394",
    "vin": "1g1pe5sb4g7136394"
}
{
    "timestamp": "2020-04-25T09:50:40.616615",
    "dealer": "simpleautos",
    "url": "https://www.simpleautos.ca//cars/used/2011-Honda-Accord-Sedan-Sedan-106066",
    "price": "8299",
    "make": "Honda",
    "model": "Accord Sedan",
    "year": "2011",
    "body_style": "Sedan",
    "mileage": "182022",
    "transmission": "5 Speed Automatic",
    "cylinders": "6",
    "displacement": "3.5",
    "drivetrain": "FWD",
  

AttributeError: 'NoneType' object has no attribute 'contents'